# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.
import numpy as np
# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = (X_train.shape[1],X_train.shape[2],X_train.shape[3])

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).size

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import random
# Visualizations will be shown in the notebook.
%matplotlib inline

def visualize(X_train, y_train) :
        index = random.randint(0, len(X_train))
        image = X_train[index].squeeze()

        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print('Label : ',y_train[index])
        
visualize(X_train, y_train)

The library member 'class Dataset' return a Dataset of features and labels by taking as input an url and the name of the repository. It offers the following features :
    1.download : download the zip file from an url
    2.uncompress : uncommpres the zip file and extract the pickle file to a given repository
    3.load : load the dataset from the pickle file in memory
    4.visualize : show randomly feature and label
    5.give statistics on dataset : 
                5.1.the number of lable or feature per class
                5.2.the number of the class in the dataset
                5.3.the shape of the feature and label
                5.4.the class which has the max number of label and feature (useful when augmenting the dataset)
    6.balance : 

In [ ]:
import os
import random
import pickle
import wget 
from urllib.request import urlretrieve
from  urllib.request import urlopen
import urllib.error 
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import cv2
import imgaug as ia
from PIL import Image
from tqdm import tqdm
from zipfile import ZipFile
from tensorflow.python.ops.variables import Variable
from itertools import groupby
from imgaug import augmenters as iaa
from enum import Enum, unique
import inspect
%matplotlib inline

print('All modules imported.')

source = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'
repository = 'traffic-signs-data.zip'
batch_count = 128

In [ ]:
class Dataset(object):
    def __init__(self,source, repository):
        
        print('Status of Initialization Step #0')
        if source is None:
            print('None source parameter is not allowed. Please, provide a source parameter.')
            return None
        if len(source)==0:
            print("Empty source parameter is not allowed. Please, provide a valid source parameter.")
            return None
        if repository is None:
            print('None repository parameter is not allowed. Please, provide a repository parameter.')
            return None
        if len(repository)==0:
            print("Empty repository parameter is not allowed. Please, provide a valid repository parameter.")
            return None
        
        
        self.__source = source
        self.__repository=repository
        self.__datacached,self.__dataloaded = False,False
        self.__downloaded , self.__uncompressed = 0,0
        self.train_size, self.test_size,self.class_size,self.label_per_class = 0,0,0,[]
        self.image_shape = None
        self.max_size_feature = None
        
    def __wget(self, source, repository):
        try:
            print(source)
            content = urlopen(source)
            repository = open(source.split('/')[-1], 'w')
            repository.write(content.read())
            content.close()
            repository.close()
        except Exception as err:
            print('ERROR:', str(err))
            raise 
        
    
    def __download(self):
        print('Download Step #1')
        #print(self.__source)
        for src, rep in tqdm(zip(self.__source, self.__repository)):
            try :
                if not os.path.isfile(rep):
                    print('Start download files from '+src+'...')
                    urlretrieve(src,rep)
                    #rep = wget.download(src)
                    #self.__wget(src,rep)
                    print('Download finished.')
                    self.__downloaded += 1
                else :
                    print('Repository path '+rep+ ' is file! We can not save\
                            files because no directory path is provided.')
            except urllib.error.URLError: #Exception as inst : #
                    print('file not found corresponding to ' + src +'. moving on...')
                    raise
                    
        if self.__downloaded != 0 :
            print('Status of download Step : ' +str(self.__downloaded) + ' files are downloaded over '+\
                      str(len(self.__source)))
            
    def __uncompress(self):
        print('Uncompression Step #3')
        destfiles=[]
        for path in self.__repository :
        #Instantiate zipfile class as zipf context
            #print(path)
            with ZipFile(path,'r') as zipf:
                #extracting file name list using progressbar
                pbfiles = zipf.namelist()
                for fname in tqdm(pbfiles, unit='files'):
                    #check whether  file name do not point to a directory. Thus do not process it 
                    if not fname.endswith('/'):
                        #unzip the image file using the file name
                        print('Unzip the image file '+fname)
                        #with zipf.open(fname) as file:
                        zipf.extract(fname)

                        # Now store the uncompressed data
                        #fdest = fname[:-3]  # remove the '.gz' from the filename
                        #print(fdest)
                        destfiles.append(fname)
                        
            print('Status of uncompression Step :  '+ str(self.__uncompressed)+ \
                      'files are uncompressed over' +str(len(pbfiles)))
        destfiles = sorted(destfiles,key=os.path.getsize, reverse=True)
        return np.array(destfiles) 
    
    def __load(self,pkfile):
        # Reload the data
        print('Data Loading Step #4')
        train_features,train_labels = None,None
        if os.path.isfile(pkfile):
            #print(pkfile)
            with open(pkfile, 'rb') as f:
              pickle_data = pickle.load(f)
              features = pickle_data['features']
              labels = pickle_data['labels']
              del pickle_data  # Free up memory
            print('Status of Data loading from pickle file Step : successful')
            self.__dataloaded = True
        else :
            print('Status of Data loading from pickle file Step : failed')
        return features,labels
    
    def deserialize(self):
        desfiles = []
        print('The dataset deserialization process is starting...')
        
        if self.__downloaded==0:
            self.__download()
            
        if self.__uncompressed == 0:
            desfiles = self.__uncompress()
            
        if len(desfiles)==2 :
            train_features, train_labels = self.__load(desfiles[0])
            test_features,test_labels = self.__load(desfiles[1])
            self.train_size = train_features.shape[0]
            self.test_size = test_features.shape[0]
            self.image_shape = (train_features.shape[1],train_features.shape[2],train_features.shape[3])
            self.class_size = np.unique(train_labels).size
            self.label_per_class = np.array([(k,len(list(v))) for k,v in groupby(y_train)])
            self.max_size_feature = max(max(self.label_per_class))
        print('The dataset deserialization process is terminated...')
        
        return train_features,train_labels, test_features,test_labels
    
    def visualize(self, X_train, y_train) :
        index = random.randint(0, len(X_train))
        image = X_train[index].squeeze()

        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print(y_train[index])
        
    
                

In [ ]:
sources = [source]
repositories = [repository]

ds = Dataset(sources, repositories)
X_train, y_train,X_test, y_test = ds.deserialize()
print("Number of training examples =", ds.train_size)
print("Number of testing examples =", ds.test_size)
print("Image data shape =", ds.image_shape)
print("Number of classes =", ds.class_size)
print("Number of label_per_class =", ds.label_per_class)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
'''
1: Data Augmentation
    This technique provides a way genrerate more data from the existing and thus makes cnn or dnn to be 
    trained on huge number of training dataset to reach better performance and accuracy.
'''

class DataAugmenter(object):
    '''
    This library class is a data augmenter pipeline manager using the Augmenters Module referenced at this url:
    https://github.com/aleju/imgaug
   '''
    class Tasks(object):
        def __str__(self):
            return str(self.value)
        
        FlipVertical = 'flip_vertical'
        FlipHorizontal = 'flip_horizontal'
        Crop = 'crop'
        GaussianBlur = 'gaussian_blur'
        AdditiveGaussianNoise = 'additive_gaussian_noise'
        Dropout = 'dropout'
        Add = 'add'
        Multiply = 'multiply'
        ContrastNormalization = 'contrast_normalization'
        Affine = 'afine'
        ElasticTransformation = 'elastic_transformation'
        
        
    def __init__(self):
        
        # random example images
        #images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)

        # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
        # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
        st = lambda aug: iaa.Sometimes(0.5, aug)
        self.pipeline = []
        # Define our sequence of augmentation steps that will be applied to every image
        # All augmenters with per_channel=0.5 will sample one value _per image_
        # in 50% of all cases. In all other cases they will sample new values
        # _per channel_.
        self.tasks = []
    
    def flip_horizontal(self):
        # horizontally flip 50% of all images
        if not "iaa.Fliplr" in self.pipeline:
            self.pipeline.append(iaa.Fliplr(0.5))
            
    def flip_vertical(self):
        # vertically flip 50% of all images
        if not "iaa.Flipud"  in self.pipeline:
            self.pipeline.append(iaa.Flipud(0.5))
            
    def crop(self):
        # crop images by crop range of 0-10%  of their height/width
        if not "iaa.Crop" in self.pipeline:
            self.pipeline.append(st(iaa.Crop(percent=(0, 0.1))))
    
    def gaussian_blur(self):
        # blur images with a sigma range between 0 and 3.0
        if not "iaa.GaussianBlur" in self.pipeline:
            self.pipeline.append(st(iaa.GaussianBlur((0, 3.0))))
    
    def additive_gaussian_noise(self):
        # add gaussian noise to images
        if "iaa.AdditiveGaussianNoise" in self.pipeline:
            self.pipeline.append(st(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.2), per_channel=0.5)))
    
    def dropout(self):
        # randomly remove up to 10% of the pixels  
        if not "iaa.Dropout" in self.pipeline:
            self.pipeline.append(st(iaa.Dropout((0.0, 0.1), per_channel=0.5)))
    
    def add(self):
        # change brightness of images (by -10 to 10 of original value)
        if not "iaa.Add" in self.pipeline:
            self.pipeline.append(st(iaa.Add((-10, 10), per_channel=0.5)))
    
    def multiply(self):
        # change brightness of images (50-150% of original value)
        if not "iaa.Multiply" in self.pipeline:
            self.pipeline.append(st(iaa.Multiply((0.5, 1.5), per_channel=0.5)))
        
    def contrast_normalization(self):
        # improve or worsen the contrast
        if not "iaa.ContrastNormalization" in self.pipeline:
            self.pipeline.append(st(iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5)))
    
    def affine(self):
        
        if not "iaa.Affine" in self.pipeline:
            self.pipeline.append(st(iaa.Affine(
                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                    translate_px={"x": (-16, 16), "y": (-16, 16)}, # translate by -16 to +16 pixels (per axis)
                    rotate=(-45, 45), # rotate by -45 to +45 degrees
                    shear=(-16, 16), # shear by -16 to +16 degrees
                    order=ia.ALL, # use any of scikit-image's interpolation methods
                    cval=(0, 1.0), # if mode is constant, use a cval between 0 and 1.0
                    mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                )))
            
    def elastic_transformation(self):
        # apply elastic transformations with random strengths
        if not "iaa.ElasticTransformation" in self.pipeline:
            self.pipeline.append(st(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)))
    
        
    def register(self,tasks):
        #Allow to register the tailored data augmentation pipeline        
        print('Augmenter Pipeline registration process is starting ...')
        if tasks is None:
            print('None tasks parameter is not allowed. Please, provide a tasks parameter.')
            return None
        if len(tasks)==0:
            print("Empty tasks parameter is not allowed. Please, provide a valid tasks parameter.")
            return None
        
        self.tasks = tasks
        
        methnames = [m for m in dir(self) if inspect.ismethod(getattr(self, m))]                     

        for taskname in tasks :                         
            if taskname not in methnames:
                print('The following method does not exists: ', taskname)
                return
        for taskname in tasks :
            self.__getattribute__(taskname)()
        
        print('Augmenter Pipeline registration process is terminated ...')
    
    def execute(self, images, show=False):
        print('Augmenter Pipeline execution process is starting ...')
        pline = iaa.Sequential(self.pipeline, 
                             random_order = True # do all of the above in random order
                                )                         
        augmented_images = pline.augment_images(images)
        
        if show :
            # show an image with 8*8 augmented versions of image 0
            seq.show_grid(images[0], cols=8, rows=8)

        print('Augmenter Pipeline execution process is terminated ...')
        
        return augmented_images



def apply_augmentation(X_train, y_train, apply=False) :
    
    da = DataAugmenter()
    
    tasks = [da.Tasks.FlipHorizontal,da.Tasks.FlipVertical,da.Tasks.Crop,
                da.Tasks.GaussianBlur,da.Tasks.Dropout,
             da.Tasks.Affine,da.Tasks.ElasticTransformation]
    
    da.register(tasks)
    
    batch_features, batch_labels  = load_batch(0,X_train, y_train)
    
    if apply :
        images_aug = da.execute(batch_features)
    else:
        images_aug = batch_features
        print('Not Apply')
    #train_on_images(images_aug)
    
    for idx,image in enumerate(images_aug):
        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print(batch_labels[idx])
        
def load_batch(batch_i,train_features,train_labels):
    batch_size = 1000 #train_features/batch_count
    # The training cycle
    #for batch_i in range(batch_count):
    # Get a batch of training features and labels
    batch_start = batch_i*batch_size
    batch_features = train_features[batch_start:batch_start + batch_size]
    batch_labels = train_labels[batch_start:batch_start + batch_size]
    return batch_features, batch_labels   
        

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**
The preprocess stage aims to :
    1.Split the train dataset into train dataset and validation datase.
    2.Normalize data

1.Split the train dataset into train dataset and validation dataset. If the number of input data is insuffisiant we can add more data using the Data augmentation process. For the purpose of this project, i propose to generate more data for the train and test datasets using the data augmenter and split randomly the augmented train dataset into a new train dataset and a validation dataset.


2.Data Normalization : in order to make the learning process performing better. The CNN learns its weights by adding gradient error vectors multiplied by the learning rate computed from backprop to weight matrices as the training process is running. so we end up with the following equation :
W = W + (learningrate * gradient descent)

Initial conditions : At the begining of the learning process, the input features values distributions are different for each one (feature). Thus the learning rate would correct in each dimension from one another. it might be over compensating a correction in one weight dimension while undercompensating in another.

The gradient descent find itself oscillating and unable to center or moving slowly  onto a better minima. So therefore, to reach a better minima and avoiding the oscillation (see above)  we need to normalize the input features values.

The scenario for preprocessing dataset is based on dataset normalization using three following methods to compare the average error (%) for each (http://ijssst.info/Vol-15/No-2/data/3251a024.pdf §IV. RESULTS AND DISCUSSIONS.)

The methods are :
    1.Simple Rescale
    2.Substract Mean 
    3.Feature Standardization


Each normalization methods is implemented by a "normalized" pipeline (see "class DataPreProcessor" bellow).
The normalization is part of the preprocess stage and should be applied after the Data augmentation stage (see later "class DataAugmenter") if exists.




In [ ]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.
import os
import random
import pickle
import wget 
import inspect
from urllib.request import urlretrieve
from  urllib.request import urlopen
import urllib.error 
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm
from zipfile import ZipFile
from tensorflow.python.ops.variables import Variable
%matplotlib inline

print('All modules imported.')

### Preprocess the data here.
### Feel free to use as many code cells as needed.

class DataPreProcessor(object):
    '''
    http://cs231n.github.io/neural-networks-2/#datapre
    There are three common forms of data preprocessing a data matrix X, 
    where we will assume that X is of size [N x D] (N is the number of data, D is their dimensionality).
    '''
    
    class Tasks(object):
        SimpleRescale = '_simple_rescale'
        MeanSubstract = '_mean_substract'
        FeatureStandardization = '_feature_standardization'
        
    class Normalizer(object):
        
        def _simple_rescale(self,images):
            images /= 255
            return images
        
        def _mean_substract(self,images):
            print('_zero_centred')
            print(images.dtype)
            #np.mean(images, axis = 0,dtype=np.uint8)
            images -= np.mean(images, axis = 0,dtype=np.uint8)
            return images
        
        def _feature_standardization(self, images):
            print('_normalize')
            images /= np.std(images, axis = 0)
            return images
        
    class Sequential(object):
        
        def __init__(self, sequence):
            self.sequence = sequence
        
        def preprocess_images(self,images):
            obj = self.sequence[0]
            self.sequence.remove(obj)
            
            for seq in self.sequence :
                images = getattr(obj, seq)(images)
                
            return images
    
    def __init__(self):
        self.pipeline = []
        ppc = self.Normalizer()
        self.pipeline.append(ppc)
    
    def simple_rescale(self):
        if not "_simple_rescale" in  self.pipeline:
            self.pipeline.append("_simple_rescale") 
    
    def mean_substract(self):
        if not "_mean_substract" in  self.pipeline:
            self.pipeline.append("_mean_substract") 
        
    def feature_standardization(self):
        if "_feature_standardization" in  self.pipeline :
            self.pipeline.append("_feature_standardization") 
        
    def register(self,tasks):
        print('Preprocessor Pipeline registration process is starting ...')
        if tasks is None:
            print('None tasks parameter is not allowed. Please, provide a tasks parameter.')
            return None
        if len(tasks)==0:
            print("Empty tasks parameter is not allowed. Please, provide a valid tasks parameter.")
            return None
        
        self.tasks = tasks
        
        methnames = [m for m in dir(self) if inspect.ismethod(getattr(self, m))]                     
        print(methnames)
        print(tasks)
        for taskname in tasks :                         
            if taskname not in methnames:
                print('The following method does not exists: ', taskname)
                return
        for taskname in tasks :
            self.__getattribute__(taskname)()
        
        print('Preprocessor Pipeline registration process is terminated ...')
        
    def execute(self, images, show=False):
        print('Preprocessor Pipeline execution process is starting ...')
        pline = self.Sequential(self.pipeline)
        
        imgs = pline.preprocess_images(images)
        
        if show :
            # show an image with 8*8 augmented versions of image 0
            seq.show_grid(imgs[0], cols=8, rows=8)

        print('Preprocessor Pipeline execution process is terminated ...')
        
        return imgs

def apply_preprocessing(X_train, y_train, apply=False) :
    '''
    This method apply the normalization pipleline composed of Mean Substract and Feature standardization ==>
    two dimensions Zero centred data
    The normalization is applied on a batch of dataset.
    '''
    dpp = DataPreProcessor()
    pipeline = [dpp.Tasks.MeanSubstract,dpp.Tasks.FeatureStandardization]
    dpp.register(pipeline)
    
    batch_features, batch_labels  = load_batch(0,X_train, y_train)
    
    if apply :
        images_aug = dpp.execute(batch_features)
    else:
        images_aug = batch_features
        print('Not Apply')
    #train_on_images(images_aug)
    
    for idx,image in enumerate(images_aug):
        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print(batch_labels[idx])
        
def load_batch(batch_i,train_features,train_labels):
    batch_size = 1000 #train_features/batch_count
    # The training cycle
    #for batch_i in range(batch_count):
    # Get a batch of training features and labels
    batch_start = batch_i*batch_size
    batch_features = train_features[batch_start:batch_start + batch_size]
    batch_labels = train_labels[batch_start:batch_start + batch_size]
    return batch_features, batch_labels   

In [ ]:
apply_preprocessing(X_train, y_train,True) 

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.